In [1]:
from google.colab import files
files.upload() # kaggle.jsonをアップロード
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [2]:
# scikit-learn のSimpleImputer を利用する場合
# データの置換等の処理はプログラム3.10と同じであるため省略
# SimpleImputerクラスをインポート
from sklearn.impute import SimpleImputer
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier \
as RandomForest
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
# データの準備
def prepare():
    !kaggle datasets download -d uciml/horse-colic
    !unzip horse-colic.zip
# データを読込み
def preprocess():
    df_train = pd.read_csv('horse.csv')
    df_train = df_train.replace('NA', 'NaN')
    # 分類に使用する特徴量
    features = [ 'surgery', 'age', 
                 'rectal_temp', 'pulse',
                 'respiratory_rate', 
                 'packed_cell_volume',
                 'total_protein', 
                 'abdomo_protein',
                 'surgical_lesion',
                 'lesion_1', 'lesion_2', 'lesion_3','cp_data']
    print(len(df_train))
    X = df_train.loc[:,features].values
    y = df_train.loc[:,['outcome']].values
    return df_train, X, y, features

# 欠損値を平均値で補完
def replace_mean(df_train, features, simple_imp):
    # outcomeを0,1（died/euthanized/lived) に置換 
    mp = {'died':0, 'euthanized':1, 'lived':2}
    kys = list(mp.keys())
    df_train = df_train[df_train['outcome'].isin(kys)]
    yesno = {'no':0, 'yes':1}
    # surgery, surgical_lesion, cp_data の
    # 列の値を0,1（no/yes) に置換
    for f in ['surgery', 'surgical_lesion', 'cp_data']:
        df_train[f].replace(yesno, inplace=True)

    # ageを0,1（young/adult) に置換
    df_train['age'].replace({'young': 0, 'adult': 1},
                            inplace=True)

    # 分類に使用するデータを格納
    X_train = df_train.loc[:, features].values
    # outcomeラベルをターゲットに格納
    df_y = pd.DataFrame(
             df_train.loc[:,['outcome']].values,
             columns=['outcome'])
    y_train = df_y.replace(mp).values.ravel()



    # 欠損値(NaN)を平均値に置換
    if simple_imp == None:
        simple_imp = SimpleImputer(missing_values=np.nan,
                                 strategy='mean')
    simple_imp.fit(X_train)
    X_train = simple_imp.transform(X_train) 
    df = pd.DataFrame(X_train, columns=features)
    # 欠損値の状態を確認
    print('# of Missing Values:\n', 
            df.isnull().any(axis=1).value_counts())
    return X_train, y_train, simple_imp

def main():
    prepare() 
    df_train, X, y, features = preprocess()
    # 学習データとテストデータに分ける
    #(train:test = 7:3)
    X_train, X_test, y_train, y_test = train_test_split(
                                      X, y, 
                                      train_size=0.7, 
                                      random_state=1)
    # 学習データの欠損値補完
    fc = features + ['outcome']
    df_train = pd.DataFrame(np.concatenate(
      (X_train,y_train), axis=1), columns=fc)
    X_train, y_train, simple_imp = \
                           replace_mean(
                           df_train, 
                           features, None)
    # テストデータの欠損値補完
    # (学習データを用いて作成したsimple_impによる補完)
    fc = features + ['outcome']
    df_test = pd.DataFrame(np.concatenate(
        (X_test, y_test), axis=1), columns=fc)
    X_test, y_test, _ = replace_mean(
    df_test, features, simple_imp)
    # ランダムフォレストで学習
    model = RandomForest(n_estimators=100, 
                         random_state=1,
                         max_depth=7).fit(
                         X_train, y_train)
    print('Accuracy: {:.3f}'.format(
                       model.score(X_test, y_test)))
    y_pred = model.predict(X_test)
    target_names = ['died', 'euthanized', 'lived']
    print(classification_report(y_test, y_pred,
                             target_names=target_names))

if __name__ == '__main__':
    main()


  0% 0.00/11.0k [00:00<?, ?B/s]
100% 11.0k/11.0k [00:00<00:00, 20.7MB/s]
Archive:  horse-colic.zip
  inflating: datadict.txt            
  inflating: horse.csv               
299
# of Missing Values:
 False    209
dtype: int64
# of Missing Values:
 False    90
dtype: int64
Accuracy: 0.789
              precision    recall  f1-score   support

        died       1.00      0.54      0.70        28
  euthanized       0.60      0.33      0.43         9
       lived       0.76      1.00      0.86        53

    accuracy                           0.79        90
   macro avg       0.79      0.62      0.66        90
weighted avg       0.82      0.79      0.77        90

